In [119]:
import pandas as pd
pd.set_option('display.max_columns',None)
from datetime import timedelta
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
df=pd.read_parquet('pre_hos.parquet')
df

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,drug_warfarin,abnormal,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-23 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [3]:
df['time']=pd.to_datetime(df['time'])
df['stoptime']=pd.to_datetime(df['stoptime'])
df['intime']=pd.to_datetime(df['intime'])
df['outtime']=pd.to_datetime(df['outtime'])

In [4]:
mask = df['stoptime'].notna() & (df['stoptime'] > df['outtime'])

In [5]:
mask

0          False
1          False
2           True
3           True
4          False
           ...  
4941947    False
4941948    False
4941949    False
4941950    False
4941951    False
Length: 4941952, dtype: bool

In [6]:
df.loc[mask, 'outtime'] = df['stoptime']

In [7]:
df

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,drug_warfarin,abnormal,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 21:00:00,2159-03-23 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [8]:
first_range = df.columns[40:59]
second_range = df.columns[60:79]
all_relevant_cols = first_range.union(second_range)

In [11]:
grouped = df.groupby('hadm_id')
total_groups = grouped.ngroups 

In [12]:
progress_bar = tqdm(grouped, total=total_groups, desc="Processing groups")

for name, group in progress_bar:
    for index, row in group.iterrows():
        if row['stoptime'] is not pd.NaT:
            cols_with_ones = all_relevant_cols[row[all_relevant_cols] == 1]
            mask = (group['time'] <= row['stoptime']) & (row['time'] <= group['time'])
            for col in cols_with_ones:
                df.loc[group[mask].index, col] = 1

Processing groups: 100%|██████████| 49261/49261 [32:42<00:00, 25.10it/s]


In [15]:
df.drop_duplicates(inplace=True)

In [17]:
df=df.sort_values(by=['hadm_id','time'])
df.reset_index(drop=True,inplace=True)
df

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,drug_warfarin,abnormal,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 21:00:00,2159-03-23 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20000019.0,2159-03-21 06:45:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [18]:
#df.to_parquet('pre_hos_v0.1.parquet')

In [51]:
import pandas as pd

data = []

hadm_id_list = df_sam['hadm_id'].unique()

for hadm_id in tqdm(hadm_id_list):
    hadm_data = df_sam[df_sam['hadm_id'] == hadm_id]
    
    time_range = pd.date_range(start=hadm_data['time'].min() + pd.Timedelta(hours=6), end=hadm_data['time'].max(), freq='6H')
    
    for time in time_range:
        con_data = hadm_data[(hadm_data['time'] >= time - pd.Timedelta(hours=6)) & (hadm_data['time'] < time)]
        
        #con_data_1 = hadm_data[(hadm_data['time'] > time - pd.Timedelta(hours=2)) & (hadm_data['time'] <= time- pd.Timedelta(hours=1))]
        
        service = df.columns[40:59]
        
        drug = df.columns[60:79]
        
        lab_ab = df.columns[80]    

        lab = df.columns[81:]
        aggregated_metrics = [con_data[metric].max() for metric in service]

        aggregated_metrics_1 = [con_data[metric].max() for metric in drug]

        aggregated_metrics_2 = [con_data[metric].sum() for metric in [lab_ab]]

        aggregated_metrics_3 = [con_data[metric].max() for metric in lab] + [con_data[metric].min() for metric in lab] + [con_data[metric].mean() for metric in lab]
        
        data.append((hadm_id, time, *aggregated_metrics,*aggregated_metrics_1,*aggregated_metrics_2,*aggregated_metrics_3))

result_columns = ['hadm_id', 'time']+ list(service)+list(drug)+list(lab_ab)+['max_' + col for col in lab] + ['min_' + col for col in lab] + ['mean_' + col for col in lab]
result_df = pd.DataFrame(data, columns=result_columns)

100%|██████████| 2/2 [00:00<00:00, 36.70it/s]


ValueError: 201 columns passed, passed data had 194 columns

In [120]:
df[df['hadm_id']==20000019]

,hadm_id,time,race,gender,age,label,Comorbidity_AIDS_H1V,Comorbidity_Alcohol_abuse,Comorbidity_Blood_loss_anemia,Comorbidity_Chronic_pulmonary_disease,Comorbidity_Coagulopathy,Comorbidity_Deficiency_anemia,Comorbidity_Depression,Comorbidity_Diabetes_complicated,Comorbidity_Diabetes_uncomplicated,Comorbidity_Drug_abuse,Comorbidity_Fluid_electrolyte_disorders,Comorbidity_Hypothyroidism,Comorbidity_Liver_disease,Comorbidity_Lymphoma,Comorbidity_Metastatic_cancer,Comorbidity_Obesity,Comorbidity_Other_neurological_disorders,Comorbidity_Paralysis,Comorbidity_Pepticulcer_disease,Comorbidity_Peripheral_vascular,Comorbidity_Psychoses,Comorbidity_Pulmonary_circulation,Comorbidity_Renal_failure,Comorbidity_Rheumatoid_arthritis_collagen_vascular_diseases,Comorbidity_Solid_tumor_without_metastasis,Comorbidity_Weight_loss,Comorbidity_cardiac,Comorbidity_chf,Comorbidity_hypertension_complicated,Comorbidity_hypertension_uncomplicated,Comorbidity_valvular_disease,intime,outtime,stoptime,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,sus_infec,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,drug_warfarin,abnormal,Basophils,Eosinophils,HCT,hemoglobin,Lymphocytes,MCH,MCHC,MCV,Monocytes,Neutrophils,Platelet,RDW,Red Blood Cells,WBC,ALT,Alb,Alk_Phos,AG,AST,bicarb,T_Bil,"Calcium, Total",Cl,SCr,Gl,Mg,PHOS,Potassium,Na,BUN,INR,PT,PTT,fibrinogen,thrombin,ldh,nt_probnp,baseexcess,paco2,PH,PaO2,ck_mb,tropt,CK,lactate,crp,D_Bil,fio2,d_dimer,ggt,o2sat
0,20000019.0,2159-03-20 22:48:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 16:54:19,2159-03-21 17:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000019.0,2159-03-21 01:00:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2159-03-20 22:48:00,2159-03-23 21:00:00,2159-03-23 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20000019.0,2159-03-21 06:45:00,Other,F,76.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [56]:
import numpy as np
#import pandas as pd

data = []

hadm_id_list = df['hadm_id'].unique()
service = df.columns[40:59]        
drug = df.columns[60:79]        
lab_ab = df.columns[80]    
lab = df.columns[81:]
# 결과 컬럼 생성
result_columns = ['hadm_id', 'time'] + list(service) + list(drug) + [lab_ab] + ['max_' + col for col in lab] + ['min_' + col for col in lab] + ['mean_' + col for col in lab]

# 데이터 추가 과정에서 길이 확인 및 기본값 설정
for hadm_id in tqdm(hadm_id_list):
    hadm_data = df[df['hadm_id'] == hadm_id]
    time_range = pd.date_range(start=hadm_data['time'].min() + pd.Timedelta(hours=6), end=hadm_data['time'].max(), freq='6H')
    #print(time_range)
    for time in time_range:
        con_data = hadm_data[(hadm_data['time'] >= time - pd.Timedelta(hours=6)) & (hadm_data['time'] < time)]
        
        # 지표 계산
        aggregated_metrics = [con_data[metric].max() if not con_data[metric].empty else np.nan for metric in service]
        aggregated_metrics_1 = [con_data[metric].max() if not con_data[metric].empty else np.nan for metric in drug]
        aggregated_metrics_2 = [con_data[metric].sum() if not con_data[metric].empty else 0 for metric in [lab_ab]]
        aggregated_metrics_3 = [con_data[metric].max() if not con_data[metric].empty else np.nan for metric in lab] + \
                               [con_data[metric].min() if not con_data[metric].empty else np.nan for metric in lab] + \
                               [con_data[metric].mean() if not con_data[metric].empty else np.nan for metric in lab]
        
        # 데이터 추가
        row_data = (hadm_id, time) + tuple(aggregated_metrics) + tuple(aggregated_metrics_1) + tuple(aggregated_metrics_2) + tuple(aggregated_metrics_3)
        data.append(row_data)

# DataFrame 생성
result_df = pd.DataFrame(data, columns=result_columns)


100%|██████████| 49261/49261 [46:59<00:00, 17.47it/s]  


In [57]:
result_df.to_parquet('hos_6h.parquet')

In [67]:
result_df.iloc[:,2:41]=result_df.iloc[:,2:41].fillna(0)

In [68]:
result_df.iloc[:,42:]

,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,26.5,9.2,NaN,29.8,34.8,86.0,NaN,NaN,183.0,12.6,3.09,15.7,NaN,NaN,NaN,14.0,NaN,24.0,NaN,NaN,103.0,1.1,169.0,1.5,NaN,3.5,137.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.5,9.2,NaN,29.8,34.8,86.0,NaN,NaN,183.0,12.6,3.09,15.7,NaN,NaN,NaN,14.0,NaN,24.0,NaN,NaN,103.0,1.1,169.0,1.5,NaN,3.5,137.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.5,9.2,NaN,29.8,34.8,86.0,NaN,NaN,183.0,12.6,3.09,15.7,NaN,NaN,NaN,14.0,NaN,24.0,NaN,NaN,103.0,1.1,169.0,1.5,NaN,3.5,137.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [69]:
con=list(result_df.columns[42:])
for i in tqdm(con):
    result_df['inpute_'+i] = result_df.apply(lambda row: 1 if pd.isna(row[i]) else 0, axis=1)

 64%|██████▍   | 98/152 [13:40<07:40,  8.52s/it]C:\Users\DAHS\AppData\Local\Temp\ipykernel_6628\802202249.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df['inpute_'+i] = result_df.apply(lambda row: 1 if pd.isna(row[i]) else 0, axis=1)
 65%|██████▌   | 99/152 [13:50<07:46,  8.80s/it]C:\Users\DAHS\AppData\Local\Temp\ipykernel_6628\802202249.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df['inpute_'+i] = result_df.apply(lambda row: 1 if pd.isna(row[i]) else 0, axis=1)
 66%|██████▌   | 100/152 [13:58<07:33,

In [70]:
result_df.to_parquet('hos_6h_v1.0.parquet')

# first: ffill() second: zero padding

In [77]:
result_df[con] = result_df.groupby('hadm_id')[con].transform(lambda x: x.ffill())

In [79]:
result_df[result_df.isnull().any(axis=1)]['hadm_id'].nunique()

49261

In [87]:
result_df.fillna(0, inplace=True)

In [88]:
result_df

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [92]:
static_df = df[['hadm_id','race','gender','age','label','sus_infec']+df.columns[6:37].tolist()]

In [96]:
static_df_1 = static_df.drop_duplicates()
static_df_1.reset_index(inplace=True,drop=True)

In [97]:
df_1 = pd.merge(result_df,static_df_1,on='hadm_id',how='left')

In [98]:
df_1

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [99]:
df_1=pd.get_dummies(df_1, columns=['gender', 'race'])
df_1

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [104]:
df_1.iloc[:,380:] = df_1.iloc[:,380:].astype(int)

In [110]:
ed=pd.read_csv('pre_hos_v0.1.csv')
ed_1 = ed[['hadm_id','ED']]

In [116]:
ed_2 = ed[['hadm_id','label']]

In [117]:
#ed_2.to_csv('hos_label.csv',index=0)

In [111]:
df_2 = pd.merge(df_1,ed_1,on='hadm_id',how='left')

In [113]:
df_2.to_parquet('raw_hos_6hour.parquet')

In [118]:
df_2

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [ ]:
df_2